In [1]:
import pandas as pd
import numpy as np
import h5py

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.model_selection import GridSearchCV


In [2]:
df = pd.read_csv('DfFinal.csv')
df = df.sample(frac=1).reset_index(drop=True)
df

,Unnamed: 0,Unnamed: 0.1,type,posts
0,2433,775,ENFJ,Honestly that doesnt help me at all The only p...
1,4418,4229,ESTP,Ive actually think that once IQ do seem to be ...
2,159,3173,INFJ,https www youtube com watchv POAt jbh Z https ...
3,302,6191,INFP,my childhood be happy and fill with friends as...
4,2061,2629,INTP,Hmm I think Im quite easy to become friends wi...
...,...,...,...,...
4795,1729,7023,ISFJ,I cant relate to BBCs Sherlock but I can somew...
4796,2953,6078,ENFP,i always get call naive i want people to not t...
4797,3812,7366,ESFP,Crazy stuff I know Lets just say I do not acti...
4798,2280,6561,ISTJ,The similar joke be due to the character be wr...


In [3]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df

,type,posts
0,ENFJ,Honestly that doesnt help me at all The only p...
1,ESTP,Ive actually think that once IQ do seem to be ...
2,INFJ,https www youtube com watchv POAt jbh Z https ...
3,INFP,my childhood be happy and fill with friends as...
4,INTP,Hmm I think Im quite easy to become friends wi...
...,...,...
4795,ISFJ,I cant relate to BBCs Sherlock but I can somew...
4796,ENFP,i always get call naive i want people to not t...
4797,ESFP,Crazy stuff I know Lets just say I do not acti...
4798,ISTJ,The similar joke be due to the character be wr...


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_enc = le.fit_transform(df['type'])

In [5]:
cv = TfidfVectorizer()
X = cv.fit_transform(df.posts).toarray()

In [8]:

del df
#X= df.posts

X_ids = list(range(len(X)))

X_tr_ids, X_te_ids, y_tr, y_te = train_test_split(X_ids, y_enc, test_size=0.3)

f = h5py.File('train.h5py', 'w')
f.create_dataset(f"inputs", data=X[X_tr_ids], dtype=np.int)
f.create_dataset(f"labels", data=y_tr, dtype=np.int)
f.close()

f = h5py.File('test.h5py', 'w')
f.create_dataset(f"inputs", data=X[X_te_ids], dtype=np.int)
f.create_dataset(f"labels", data=y_te, dtype=np.int)
f.close()



In [ ]:
pip install h5py

In [6]:
print(X[22])

[0. 0. 0. ... 0. 0. 0.]


In [10]:

f = h5py.File('train.h5py', 'r')
X_train = np.array(f.get('inputs'), dtype=np.int)
Y_train = np.array(f.get('labels'), dtype=np.int)
f.close()

f = h5py.File('test.h5py', 'r')
X_test = np.array(f.get('inputs'), dtype=np.int)
Y_test = np.array(f.get('labels'), dtype=np.int)
f.close()

Y_test


array([10,  0, 13, ...,  6, 12,  8])

In [ ]:
#MLP classifier
from sklearn.neural_network import MLPClassifier
MLPCLF=MLPClassifier()
parameters={
    'alpha': [.01,.001,1,10,0.1],
    'activation': ["logistic", "relu",'tanh','identity'],
    'learning_rate' :['constant', 'invscaling', 'adaptive'],
    }
mlp_grid=GridSearchCV(MLPCLF,param_grid=parameters,verbose=10,scoring='accuracy',cv=2)
mlp_grid.fit(X_train,Y_train)

import pickle

with open('SAVE_GRIDSEARCH.obj', 'wb') as save_my_ass:
    pickle.dump(clf, save_my_ass)


Fitting 2 folds for each of 60 candidates, totalling 120 fits
[CV 1/2; 1/60] START activation=logistic, alpha=0.01, learning_rate=constant....
[CV 1/2; 1/60] END activation=logistic, alpha=0.01, learning_rate=constant; total time= 4.8min
[CV 2/2; 1/60] START activation=logistic, alpha=0.01, learning_rate=constant....
[CV 2/2; 1/60] END activation=logistic, alpha=0.01, learning_rate=constant; total time= 4.8min
[CV 1/2; 2/60] START activation=logistic, alpha=0.01, learning_rate=invscaling..
[CV 1/2; 2/60] END activation=logistic, alpha=0.01, learning_rate=invscaling; total time= 4.2min
[CV 2/2; 2/60] START activation=logistic, alpha=0.01, learning_rate=invscaling..


In [ ]:
#Remove the ipykernel using conda remove ipykernel
#and then resinstall with lower version with pip install ipykernel ==4.2.2